In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale, robust_scale
import numpy as np
import torch.nn.functional as F

data = pd.read_csv('./train.csv', header='infer', index_col=0)
datatest = pd.read_csv('./test.csv', header='infer', index_col=0)

class Aset(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.X = data.drop('label', axis=1)
        self.y = data['label']

    def __getitem__(self, index):
        X = self.X.iloc[index].to_numpy()
        y = self.y.iloc[index]
        return X, y
    
    def __len__(self):
        return len(self.X)

import torch
import torch.nn as nn

import torch
import torch.nn as nn

class CNN1D(nn.Module):
  def __init__(self):
    super(CNN1D, self).__init__()
    # 输入数据为280维
    self.conv1 = nn.Conv1d(1, 16, 5)
    # 16 * 276
    self.pool1 = nn.MaxPool1d(2)
    # 16 * 138
    self.conv2 = nn.Conv1d(16, 32, 5)
    # 32 * 134
    self.pool2 = nn.MaxPool1d(2)
    # 32 * 67
    self.conv3 = nn.Conv1d(32, 64, 5)
    # 64 * 63
    self.pool3 = nn.MaxPool1d(3)
    # 64 * 21
    self.fc1 = nn.Linear(64 * 21, 128)
    self.fc2 = nn.Linear(128, 2)

  def forward(self, x):
    # print(x.shape)
    x = F.relu(self.conv1(x.unsqueeze(1)))
    # print(x.shape)
    x = self.pool1(x)
    # print(x.shape)
    x = F.relu(self.conv2(x)) 
    # print(x.shape)
    x = self.pool2(x)
    # print(x.shape)
    x = F.relu(self.conv3(x))
    # print(x.shape)
    x = self.pool3(x)
    # print(x.shape)
    x = x.view(-1, 64*21)
    # print(x.shape)
    x = F.relu(self.fc1(x))
    # print(x.shape)
    x = self.fc2(x)
    return x

class LstmRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, output_size=3, num_layers=2):
        super().__init__()
 
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 32)
        self.fc2 = nn.Linear(32, output_size)
        

        # nn.init.xavier_normal(self.fc1.weight)
        # nn.init.xavier_normal(self.fc2.weight)
 
    def forward(self, inputs):
        # print(inputs.shape)
        inputs = inputs.unsqueeze(2)
        # print(inputs.shape)
        outputs, hidden = self.rnn(inputs)  
        # print(outputs.shape)
        # print(hidden.shape)
        features = hidden[-1, :, :]
        # print(features.shape)
        outputs = self.fc1(features)
        outputs = F.relu(outputs)
        outputs = self.fc2(outputs)
        return outputs

class Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_size=128):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # self.fc3 = nn.Linear(hidden_size, hidden_size)
        # self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_dim)

    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
        

train_set = Aset(data)
train_loader = DataLoader(train_set, 32)

test_set = Aset(datatest)
test_loader = DataLoader(test_set, 32)

In [ ]:
model = LstmRNN(1, output_size=2).cuda()
torch.save(model, "model.pth")

model = torch.load("model.pth")

In [ ]:
from torch.autograd import Variable

# model = CNN1D().cuda()
# model = Model(280, 2).cuda()
model = LstmRNN(1, output_size=2).cuda()
print(model)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.8, weight_decay=1e-8)
from torch.optim.lr_scheduler import MultiStepLR
scheduler = MultiStepLR(optimizer, milestones=[20, 50], gamma=0.1)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()
    for e in range(200):
        acc = 0
        for x, y in train_loader:
            x = Variable(x.float(), requires_grad=True).cuda()
            y = Variable(y.long()).cuda()
            
            preds = model(x)
            loss = criterion(preds, y)
            loss.backward()

            pred = preds.argmax(dim=1).cuda()
            correct = (pred == y).sum()
            acc += correct.item()

            optimizer.step()
            optimizer.zero_grad()
        # scheduler.step()
        print(e, acc, acc/(len(train_loader)*32), end='\n')

train()

In [ ]:

model.eval()
res = []
for x, y in test_loader:
    preds = model(x.cuda().float())
    # print(preds)
    pred = preds.argmax(dim=1).cpu().long()
    # print(pred)
    o = list(pred.numpy())
    res = res + o

print(res)

In [ ]:
finalaa = pd.DataFrame([], columns=['name', 'label', 'num'])
import os
start = 0
for path, dir, files in os.walk('./电动车充电识别-数据/test/'):
    # print(len(files))
    for file in files:
        label: np.ndarray = res[start]
        start+=1
        num = int(str(file).split('.')[0])
        finalaa.loc[len(finalaa)] = [str(file), label, num]

finalaa.set_index('name', inplace=True)
final = finalaa.sort_values(by='num')
final = final.drop('num', axis=1)
print(final['label'].to_numpy().flatten())

final.to_csv('./resnn.csv')